# Data Prepartion - IMDb Dataset

This notebook prepares the IMDb files chosen for analysis to be merged with data from other sources. The result will be a sngle dataframe saved to disk as a CSV.

## Data Selection

The IMDb title basics and title ratings files have been chosen for further analysis. The basics file contains two relevant features for analysis in genres and runtime, as well as movie title and year. The ratings file contains two worthwhile targets in average rating and number of votes. These metrics are interesting for further investigation of their relevance to the problem of what movies Microsoft's new studio should make.

In [40]:
# import the relevant packages and load the files
import pandas as pd

basics = pd.read_csv('./zippedData/imdb.title.basics.csv.gz')
ratings = pd.read_csv('./zippedData/imdb.title.ratings.csv.gz')

In [41]:
# check to make sure the data loaded as expected
basics.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [42]:
# check to make sure the data loaded as expected
ratings.head()

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


## Join the DataFrames

The two DataFrames will be joined on the `tconst` field, a unique identifier for each movie title.

In [43]:
# check the length of tconst
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [44]:
# check the length of tconst
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


In [45]:
# check if each tconst is distinct
len(basics['tconst'].unique())

146144

In [46]:
# check if each tconst is distinct
len(ratings['tconst'].unique())

73856

There are fewer records of movies in the ratings table than the basics table. Both tables have completely distinct records. These tables should be **outer joined** to preserve information about movies that don't have an IMDb rating. *This information could still be useful after the merge with data from other sources.*

In [47]:
# set the index to the tconst column to join them
imdb = basics.set_index('tconst').join(ratings.set_index('tconst'), how='outer')

# check to see if join went as expected
imdb.head()

,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
tconst,,,,,,,
tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0
tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0
tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0
tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0
tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0


In [48]:
# check the number of records and if there are missing values for averagerating and numvotes
imdb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146144 entries, tt0063540 to tt9916754
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   primary_title    146144 non-null  object 
 1   original_title   146123 non-null  object 
 2   start_year       146144 non-null  int64  
 3   runtime_minutes  114405 non-null  float64
 4   genres           140736 non-null  object 
 5   averagerating    73856 non-null   float64
 6   numvotes         73856 non-null   float64
dtypes: float64(3), int64(1), object(3)
memory usage: 8.9+ MB


## Create new ID

The tables have been merged. All records remain intact, while there are some expected missing values for the averagerating and numvotes columns. Currently, the index is set to an id specified by the `tconst` column. In order to join this IMDb data with other sources, a new ID will need to be created. `tconst` is unique to the IMDb data. The new ID will be a string containing the name of the movie and its release year.

In [49]:
# take a look at the current format
imdb.head()

,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
tconst,,,,,,,
tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0
tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0
tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0
tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0
tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0


In [50]:
# create new column and remove any leading or trailing whitespace in the title
imdb['movie_id'] = imdb['primary_title'].str.strip()

# convert titles to lowercase
imdb['movie_id'] = imdb['movie_id'].str.lower()

# check result
imdb.head()

,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,movie_id
tconst,,,,,,,,
tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,sunghursh
tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0,one day before the rainy season
tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0,the other side of the wind
tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0,sabse bada sukh
tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0,the wandering soap opera


In [51]:
# append the year to the movie_id
imdb['movie_id'] = imdb['movie_id'] + imdb['start_year'].astype(str)

# check result
imdb.head()

,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,movie_id
tconst,,,,,,,,
tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,sunghursh2013
tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0,one day before the rainy season2019
tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0,the other side of the wind2018
tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0,sabse bada sukh2018
tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0,the wandering soap opera2017


In [52]:
# set the index to the new movie_id
imdb = imdb.set_index('movie_id')

#check result
imdb.head()

,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
movie_id,,,,,,,
sunghursh2013,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0
one day before the rainy season2019,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0
the other side of the wind2018,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0
sabse bada sukh2018,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0
the wandering soap opera2017,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0


## Drop unnecessary columns

The new index has been created, consisting of the lowercase name of the movie with the year appended at the end. The IMDb data is almost ready to be joined with other data sources. Now that the year has been included in the index, it is no longer needed for analysis. Let's drop that column. The original_title column will also be dropped as only the primary_title is necessary for further analysis. The original_title is the title in its original language.

In [53]:
# drop the original_title column, but keep all rows
imdb = imdb.drop(['original_title', 'start_year'], axis=1)

# check the result
imdb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146144 entries, sunghursh2013 to chico albuquerque - revelações2013
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   primary_title    146144 non-null  object 
 1   runtime_minutes  114405 non-null  float64
 2   genres           140736 non-null  object 
 3   averagerating    73856 non-null   float64
 4   numvotes         73856 non-null   float64
dtypes: float64(3), object(2)
memory usage: 6.7+ MB


In [54]:
# take a look
imdb.head()

,primary_title,runtime_minutes,genres,averagerating,numvotes
movie_id,,,,,
sunghursh2013,Sunghursh,175.0,"Action,Crime,Drama",7.0,77.0
one day before the rainy season2019,One Day Before the Rainy Season,114.0,"Biography,Drama",7.2,43.0
the other side of the wind2018,The Other Side of the Wind,122.0,Drama,6.9,4517.0
sabse bada sukh2018,Sabse Bada Sukh,NaN,"Comedy,Drama",6.1,13.0
the wandering soap opera2017,The Wandering Soap Opera,80.0,"Comedy,Drama,Fantasy",6.5,119.0


## Save as a CSV file

In [55]:
# save as csv file
imdb.to_csv('imdb_data')

## Summary

The IMDb data is now ready to be joined with data from other sources. The two original tables have been joined, a new movie id index has been created, and columns unnecessary for further analysis have been dropped. With the new csv file, we'll next attempt to join other data on the new movie id index.


# Matthew's EDA


Specifically mention purpose

**Stakeholder**: Microsoft's new movie studio head 

**Problem**: What type of films should the studio create? 

**Subproblem**: What films are doing "the best at the box office?" How can this studio compete with original video content from other "big companies?"

In [1]:
# Import packages for exploration
import pandas as pd
import numpy as np


## Rotten Tomatoes

Rotten tomatoes is a review-aggregation website and database for movies.

In [3]:
# This file is a TSV so the seperator should be set to '\t' and it is encoded using 'latin-1'
rt_reviews = pd.read_csv('../Data/zippedData/rt.reviews.tsv.gz', sep='\t', encoding='latin-1')
rt_movie_info = pd.read_csv('../Data/zippedData/rt.movie_info.tsv.gz', sep='\t', encoding='latin-1')


### Movie Info


In [4]:
# Visually confirming table was read properly
rt_movie_info.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [5]:
# Making observations based on table metadata
rt_movie_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


**Notes:**  
1. Columns 8, 9, and 11 seem to contain significantly more NaN values then the other columns.
2. Rating is the MPAA rating
3. 'theater_date' and 'dvd_date' not datetime objects
4. 'box_Office' appears to be the money made at the box office, but is not an integer/float
5. There is NO name assosiated with this table. Movies seem to be identified by 'id'


### REVIEWS


In [6]:
# Visually confirming table was read properly
rt_reviews.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [7]:
# Making observations based on table metadata
rt_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


**Notes:**  
1. ID seems to be a foreign key in this table. There is still no way to identify the NAME of the movie outside of speculation, though it may be irrelevant.
2. Top critic appears to be a boolean value.
3. Rating seems to be a fraction out of 5. Will need to convert to float.
4. 'Data' column not 'datetime' object.

## Final Rotten Tomato Notes

There is a great deal of useful statistics in here. If the two Rotten Tomato tables are joined a great deal of observations could be made.

1. Critic ratings based on genre  
2. Box office earnings by rating (MPAA)  
3. Rating by Runtime  
4. Writers / Directors who result in the most box office   revenue  

These are all examples of statistics that could be used to recommend actions to the head of the studio like:

1. What kind of Genre is most likely to Rate well
2. What kind of movie rating (MPAA) is most likley to make the most box office earnings
3. The target length for a movie to get the highest rating.
4. What kind of writers or directors to hire.

**Limitiations**
- The box office column has a severe amount of null values. About 80% of the entries are missing their 'box office' value.

Even without the box office column, this dataset could provide plenty of other reccomendations with the other data.


### The Movie DB

The Movie DB is a user editable database for movies and TV shows.

In [20]:
# Open and assign data to a dataframe.
tmdb = pd.read_csv('../Data/zippedData/tmdb.movies.csv.gz')

In [21]:
# Visually confirm table was read properly
tmdb.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [22]:
# There appears to be an index column already in this table. I will drop the column rather then re-read the file.
tmdb.drop(axis= 1, labels='Unnamed: 0', inplace=True)

#Re-check table
tmdb.head()

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [23]:
tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genre_ids          26517 non-null  object 
 1   id                 26517 non-null  int64  
 2   original_language  26517 non-null  object 
 3   original_title     26517 non-null  object 
 4   popularity         26517 non-null  float64
 5   release_date       26517 non-null  object 
 6   title              26517 non-null  object 
 7   vote_average       26517 non-null  float64
 8   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 1.8+ MB


**Notes:**  
1. No null values.
2. Genre_ids looks like a list of numbers that corrosponds to a genre.
3. vote_average and vote_count looks to be about user ratings.
4. Immedietly it may be intresting to look at the relationship between 'popularity' and 'vote_average'

[Popularity](https://developers.themoviedb.org/3/getting-started/popularity) seems to be a unique metric for calculating how popular an entry is based on recent user interaction with the entry 

In [24]:
# Lets look at some general stats for the integer columns
tmdb.describe()

,id,popularity,vote_average,vote_count
count,26517.000000,26517.000000,26517.000000,26517.000000
mean,295050.153260,3.130912,5.991281,194.224837
std,153661.615648,4.355229,1.852946,960.961095
min,27.000000,0.600000,0.000000,1.000000
25%,157851.000000,0.600000,5.000000,2.000000
50%,309581.000000,1.374000,6.000000,5.000000
75%,419542.000000,3.694000,7.000000,28.000000
max,608444.000000,80.773000,10.000000,22186.000000


In [25]:
# The 'popularity' mean value looks a little off to me. Let's check the column for irregularities.
tmdb['popularity'].value_counts()

0.600     7037
1.400      649
0.840      587
0.624      104
0.625       92
          ... 
3.742        1
14.749       1
7.924        1
8.414        1
9.060        1
Name: popularity, Length: 7425, dtype: int64

**Notes**  
It seems to me that '0.600' may be a placeholder value due to the sheer number of occurances.

In [26]:
# The 'popularity' column has made me suspicous, so now lets check the 'vote_count' column
tmdb['vote_count'].value_counts()

1       6541
2       3044
3       1757
4       1347
5        969
        ... 
2328       1
6538       1
489        1
2600       1
2049       1
Name: vote_count, Length: 1693, dtype: int64

In [28]:
# It is certainly possible that there are 6541 movies with 1 vote, but it feels unlikely.
tmdb[tmdb['vote_count'] == 1]

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
770,"[28, 80, 18, 53]",51488,en,Full Love,2.288,2010-01-01,Full Love,10.0,1
873,"[878, 27]",27485,en,Megaconda,1.960,2010-01-01,Megaconda,7.0,1
1004,"[12, 80]",76747,ru,Burning Daylight,1.588,2010-11-14,Burning Daylight,6.0,1
1008,"[16, 10751]",52272,en,Bratz: Pampered Petz,1.579,2010-10-05,Bratz: Pampered Petz,5.0,1
1063,"[9648, 53]",295682,en,Bright Falls,1.400,2010-04-27,Bright Falls,9.0,1
...,...,...,...,...,...,...,...,...,...
26512,"[27, 18]",488143,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1
26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1
26515,"[10751, 12, 28]",366854,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1


**Notes**  
Multiple issues discovered here, vote_average, vote_count and popularity all seem to have placeholder values.

## Final The Movie DB Notes

**Example Observations:**

1. User rating (vote_average) based on genre
2. Most common words in the title of the movies with the highest user rating (vote_average)

These are all examples of statistics that could be used to recommend actions to the head of the studio like:

1. What kind of film genre is most likely to rate well
2. What kind of words should be put in the movies title

**Limitiations**
- The box office column has a severe amount of null values. About 80% of the entries are missing their 'box office' value.
- The 'vote_count', 'vote_average', and 'popularity' all have some form of placeholder values.

The data here would require heavy cleaning, but is far from unusable.

# Matthew's Data Cleaning

**In this notebook I will be cleaning "The Movie DB" dataset (tmdb.movies.csv.gz)  for it to be merged into the master table**  

After talking with the group it was decided that we would merge our tables together into a "master table" using what we decided were [relevant columns](https://docs.google.com/spreadsheets/d/1FrTlLWIb5fVswDBTqliIks8QU7fY97RTtus5--7r_q4/edit#gid=596077008). Of the two datasets I explored, it was decided that we would keep data from "The Movie DB" dataset. The kept columns from the table will be:

**Keeping**

- release_date
- original_title
- vote_average
- vote_count
- popularity 

**Creating**

- movie_id
- release_month

and a column called "movie_id" will be created by combining the 'original_title' and the year from the'release_date' column. This column will be used as a key when creating the master 'table. release_month' column will be used in future analysis.

In [160]:
# Import packages for Cleaning
import pandas as pd
import numpy as np

In [161]:
# There is an index column in this data so I set the parameter 'index_col' to 1.
tmdb = pd.read_csv('../Data/zippedData/tmdb.movies.csv.gz', index_col=0)

In [162]:
# Visually confirm no issues with import.
tmdb.head()

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


## Selecting Desired Columns

In [163]:
# First, I'm going to drop the columns that we decided not to use: 'genre_ids', 'id', and 'original_language'
tmdb.drop(columns=['genre_ids', 'id','original_language'], inplace=True)

In [164]:
# Before dropping one of them, I'm going to check the differences between 'original_title' and 'title'
tmdb[tmdb['original_title'] != tmdb['title']]

,original_title,popularity,release_date,title,vote_average,vote_count
14,Saw 3D,20.370,2010-10-28,Saw: The Final Chapter,6.0,1488
49,Tres metros sobre el cielo,13.721,2010-12-20,Three Steps Above Heaven,7.5,960
67,Arthur 3: la guerre des deux mondes,12.679,2010-08-22,Arthur 3: The War of the Two Worlds,5.6,865
70,El secreto de sus ojos,12.531,2010-04-16,The Secret in Their Eyes,7.9,1141
75,サマーウォーズ,12.275,2010-10-13,Summer Wars,7.5,447
...,...,...,...,...,...,...
26409,你好，之华,0.600,2018-11-09,Last Letter,6.0,1
26422,El verano del león eléctrico,0.600,2018-11-12,The Summer of the Electric Lion,6.0,1
26432,Contes de Juillet,0.600,2018-03-09,July Tales,6.0,1
26494,La última virgen,0.600,2018-05-26,The Last Virgin,2.0,1


It looks like 'original_title' is either  
1. The title of the movie in its original language
2. The 'prototype' or 'working' title for the movie

I'm going to keep the 'title' column instead, as keeping the 'original_title' column will create problems when creating the 'movie_id' column later.

In [165]:
tmdb.drop(columns='original_title', inplace=True)

# Visually confirm expected result
tmdb.head()

,popularity,release_date,title,vote_average,vote_count
0,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,28.005,1995-11-22,Toy Story,7.9,10174
4,27.920,2010-07-16,Inception,8.3,22186


## Confirming Valid Data

I want to make sure that I don't have any placeholder data in my columns before sending it off to the master table.

### Popularity

In [166]:
# Checking popularity
tmdb['popularity'].value_counts()

0.600     7037
1.400      649
0.840      587
0.624      104
0.625       92
          ... 
3.742        1
14.749       1
7.924        1
8.414        1
9.060        1
Name: popularity, Length: 7425, dtype: int64

After discussion with my group, we decided that it is likley that the value '0.600' is a result of a quirk with The Movie DB's algorithm for calculating popularity. The odd results are more likley due to low user interaction. We decided to not do anything with it since its not technically placeholder data. 

### Release Date

In [167]:
tmdb['release_date'].value_counts()

2010-01-01    269
2011-01-01    200
2012-01-01    155
2014-01-01    155
2013-01-01    145
             ... 
1985-08-30      1
2012-06-28      1
1966-08-24      1
2011-02-27      1
2010-01-03      1
Name: release_date, Length: 3433, dtype: int64

There is a strong possibility that January 1st is a placeholder month and date. This won't be a problem for creating the 'movie_id' column since the year is probably correct, but it will be a problem when comparing the release month/day to anything. There are around 1000 of them out of 3000 entries.

### Title Column

In [168]:
tmdb['title'].value_counts()

Home                                     7
Eden                                     7
Aftermath                                5
Lucky                                    5
Truth or Dare                            5
                                        ..
Game of Thrones: Conquest & Rebellion    1
My Neighbourhood                         1
Erratum                                  1
A Path Appears                           1
James Gandolfini: Tribute To A Friend    1
Name: title, Length: 24688, dtype: int64

Some common names here, nothing stirkes me as odd here.

### Vote Average

In [169]:
tmdb['vote_average'].value_counts()

6.0     1940
7.0     1560
5.0     1486
10.0    1252
8.0     1231
        ... 
9.4        6
1.2        3
1.4        3
9.1        2
9.7        2
Name: vote_average, Length: 91, dtype: int64

Some of these values look odd to me, but none of them seem impossible.

### Vote Count

In [170]:
tmdb['vote_count'].value_counts()

1       6541
2       3044
3       1757
4       1347
5        969
        ... 
2328       1
6538       1
489        1
2600       1
2049       1
Name: vote_count, Length: 1693, dtype: int64

Again, nothing terribly odd here.

## Movie ID Creation

In [171]:
# This column is the lowercase name of the movie, combined with the year released at the end.
tmdb['movie_id'] = tmdb['title'].str.strip().str.lower()

#Now combine the year
tmdb['movie_id'] = tmdb['movie_id']+tmdb['release_date'].str.slice(0, 4)

In [172]:
# Visually confirm expected result.
tmdb.head()

,popularity,release_date,title,vote_average,vote_count,movie_id
0,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788,harry potter and the deathly hallows: part 12010
1,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,how to train your dragon2010
2,28.515,2010-05-07,Iron Man 2,6.8,12368,iron man 22010
3,28.005,1995-11-22,Toy Story,7.9,10174,toy story1995
4,27.920,2010-07-16,Inception,8.3,22186,inception2010


## Setting Datatypes

In [173]:
# Check datatypes
tmdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26517 entries, 0 to 26516
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   popularity    26517 non-null  float64
 1   release_date  26517 non-null  object 
 2   title         26517 non-null  object 
 3   vote_average  26517 non-null  float64
 4   vote_count    26517 non-null  int64  
 5   movie_id      26517 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 1.4+ MB


In [174]:
# All datatypes are as I need them to be except for 'release_date'. It needs to be a datetime object.
tmdb['release_date'] = tmdb['release_date'].astype('datetime64')
# Confirm change
tmdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26517 entries, 0 to 26516
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   popularity    26517 non-null  float64       
 1   release_date  26517 non-null  datetime64[ns]
 2   title         26517 non-null  object        
 3   vote_average  26517 non-null  float64       
 4   vote_count    26517 non-null  int64         
 5   movie_id      26517 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 1.4+ MB


In [175]:
# One more confirmation that I can get the month from the dt object, as it will be needed to create 'release_month'
tmdb['release_date'].dt.month

0        11
1         3
2         5
3        11
4         7
         ..
26512    10
26513     5
26514    10
26515     6
26516    10
Name: release_date, Length: 26517, dtype: int64

All other Columns are already the desired datatype.

## Release Month Creation


In [176]:
# Now that 'release_date' has been converted to a datetime object we can create the 'release_month column'
tmdb['release_month'] = tmdb['release_date'].dt.month

In [177]:
# One final check that data looks as expected
tmdb.head()

,popularity,release_date,title,vote_average,vote_count,movie_id,release_month
0,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788,harry potter and the deathly hallows: part 12010,11
1,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,how to train your dragon2010,3
2,28.515,2010-05-07,Iron Man 2,6.8,12368,iron man 22010,5
3,28.005,1995-11-22,Toy Story,7.9,10174,toy story1995,11
4,27.920,2010-07-16,Inception,8.3,22186,inception2010,7


In [178]:
# Set index as movie_id for master table.
tmdb.set_index('movie_id')

,popularity,release_date,title,vote_average,vote_count,release_month
movie_id,,,,,,
harry potter and the deathly hallows: part 12010,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788,11
how to train your dragon2010,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,3
iron man 22010,28.515,2010-05-07,Iron Man 2,6.8,12368,5
toy story1995,28.005,1995-11-22,Toy Story,7.9,10174,11
inception2010,27.920,2010-07-16,Inception,8.3,22186,7
...,...,...,...,...,...,...
laboratory conditions2018,0.600,2018-10-13,Laboratory Conditions,0.0,1,10
_exhibit_84xxx_2018,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1,5
the last one2018,0.600,2018-10-01,The Last One,0.0,1,10


In [179]:
# Finally save the file as a CSV for easier sharing
tmdb.to_csv('tmdb_data')

# Data Prepartion - IMDb Dataset

This notebook prepares the IMDb files chosen for analysis to be merged with data from other sources. The result will be a sngle dataframe saved to disk as a CSV.

## Data Selection

The IMDb title basics and title ratings files have been chosen for further analysis. The basics file contains two relevant features for analysis in genres and runtime, as well as movie title and year. The ratings file contains two worthwhile targets in average rating and number of votes. These metrics are interesting for further investigation of their relevance to the problem of what movies Microsoft's new studio should make.

In [40]:
# import the relevant packages and load the files
import pandas as pd

basics = pd.read_csv('./zippedData/imdb.title.basics.csv.gz')
ratings = pd.read_csv('./zippedData/imdb.title.ratings.csv.gz')

In [41]:
# check to make sure the data loaded as expected
basics.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [42]:
# check to make sure the data loaded as expected
ratings.head()

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


## Join the DataFrames

The two DataFrames will be joined on the `tconst` field, a unique identifier for each movie title.

In [43]:
# check the length of tconst
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [44]:
# check the length of tconst
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


In [45]:
# check if each tconst is distinct
len(basics['tconst'].unique())

146144

In [46]:
# check if each tconst is distinct
len(ratings['tconst'].unique())

73856

There are fewer records of movies in the ratings table than the basics table. Both tables have completely distinct records. These tables should be **outer joined** to preserve information about movies that don't have an IMDb rating. *This information could still be useful after the merge with data from other sources.*

In [47]:
# set the index to the tconst column to join them
imdb = basics.set_index('tconst').join(ratings.set_index('tconst'), how='outer')

# check to see if join went as expected
imdb.head()

,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
tconst,,,,,,,
tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0
tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0
tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0
tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0
tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0


In [48]:
# check the number of records and if there are missing values for averagerating and numvotes
imdb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146144 entries, tt0063540 to tt9916754
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   primary_title    146144 non-null  object 
 1   original_title   146123 non-null  object 
 2   start_year       146144 non-null  int64  
 3   runtime_minutes  114405 non-null  float64
 4   genres           140736 non-null  object 
 5   averagerating    73856 non-null   float64
 6   numvotes         73856 non-null   float64
dtypes: float64(3), int64(1), object(3)
memory usage: 8.9+ MB


## Create new ID

The tables have been merged. All records remain intact, while there are some expected missing values for the averagerating and numvotes columns. Currently, the index is set to an id specified by the `tconst` column. In order to join this IMDb data with other sources, a new ID will need to be created. `tconst` is unique to the IMDb data. The new ID will be a string containing the name of the movie and its release year.

In [49]:
# take a look at the current format
imdb.head()

,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
tconst,,,,,,,
tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0
tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0
tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0
tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0
tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0


In [50]:
# create new column and remove any leading or trailing whitespace in the title
imdb['movie_id'] = imdb['primary_title'].str.strip()

# convert titles to lowercase
imdb['movie_id'] = imdb['movie_id'].str.lower()

# check result
imdb.head()

,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,movie_id
tconst,,,,,,,,
tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,sunghursh
tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0,one day before the rainy season
tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0,the other side of the wind
tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0,sabse bada sukh
tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0,the wandering soap opera


In [51]:
# append the year to the movie_id
imdb['movie_id'] = imdb['movie_id'] + imdb['start_year'].astype(str)

# check result
imdb.head()

,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,movie_id
tconst,,,,,,,,
tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,sunghursh2013
tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0,one day before the rainy season2019
tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0,the other side of the wind2018
tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0,sabse bada sukh2018
tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0,the wandering soap opera2017


In [52]:
# set the index to the new movie_id
imdb = imdb.set_index('movie_id')

#check result
imdb.head()

,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
movie_id,,,,,,,
sunghursh2013,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0
one day before the rainy season2019,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0
the other side of the wind2018,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0
sabse bada sukh2018,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0
the wandering soap opera2017,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0


## Drop unnecessary columns

The new index has been created, consisting of the lowercase name of the movie with the year appended at the end. The IMDb data is almost ready to be joined with other data sources. Now that the year has been included in the index, it is no longer needed for analysis. Let's drop that column. The original_title column will also be dropped as only the primary_title is necessary for further analysis. The original_title is the title in its original language.

In [53]:
# drop the original_title column, but keep all rows
imdb = imdb.drop(['original_title', 'start_year'], axis=1)

# check the result
imdb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146144 entries, sunghursh2013 to chico albuquerque - revelações2013
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   primary_title    146144 non-null  object 
 1   runtime_minutes  114405 non-null  float64
 2   genres           140736 non-null  object 
 3   averagerating    73856 non-null   float64
 4   numvotes         73856 non-null   float64
dtypes: float64(3), object(2)
memory usage: 6.7+ MB


In [54]:
# take a look
imdb.head()

,primary_title,runtime_minutes,genres,averagerating,numvotes
movie_id,,,,,
sunghursh2013,Sunghursh,175.0,"Action,Crime,Drama",7.0,77.0
one day before the rainy season2019,One Day Before the Rainy Season,114.0,"Biography,Drama",7.2,43.0
the other side of the wind2018,The Other Side of the Wind,122.0,Drama,6.9,4517.0
sabse bada sukh2018,Sabse Bada Sukh,NaN,"Comedy,Drama",6.1,13.0
the wandering soap opera2017,The Wandering Soap Opera,80.0,"Comedy,Drama,Fantasy",6.5,119.0


## Save as a CSV file

In [55]:
# save as csv file
imdb.to_csv('imdb_data')

## Summary

The IMDb data is now ready to be joined with data from other sources. The two original tables have been joined, a new movie id index has been created, and columns unnecessary for further analysis have been dropped. With the new csv file, we'll next attempt to join other data on the new movie id index.